# Tokenize the notebook source code

In [80]:
import numpy as np
import pandas as pd
import tokenize
from io import BytesIO

In [3]:
df = pd.read_json("../data/all-notebooks.json",orient="index")

In [5]:
df.head()

,cell_type,source,filename,competition
0,code,from subprocess import check_output\n import n...,1806927,favorita-grocery-sales-forecasting
1,code,holiday = pd.read_csv('../input/holidays_event...,1806927,favorita-grocery-sales-forecasting
2,code,# converting date into datetime format\n holid...,1806927,favorita-grocery-sales-forecasting
3,code,# all bridge-type has puente (puente actually ...,1806927,favorita-grocery-sales-forecasting
4,code,"# for Regional and Local holidays, \n # name l...",1806927,favorita-grocery-sales-forecasting


In [117]:
def tokenize_code(code, cell_type):
    # if markdown or raw, split with " "
    if cell_type != "code":
        return []
    
    
    tokenized_code = []
    tokens = []
    
    try:
        tokens = tokenize.tokenize(BytesIO(code.encode()).readline)
    except (SyntaxError, tokenize.TokenError, IndentationError, AttributeError):
        return []
    try:
#       tokens is a generator function, so we need to also catch exceptions when calling it
        for tok in tokens:
            ret = ""
            # first token is always utf-8, ignore it
            if (tok.string == "utf-8"):
                continue
            # type 4 is NEWLINE
            elif (tok.type == 4 or tok.type == 61):
                ret = "[NEWLINE]"
            # type 5 is INDENT
            elif (tok.type == 5):
                ret = "[INDENT]"
            else:
                ret = tok.string
    #         print(tok)
    #         print(f"Type: {tok.exact_type}\nString: {tok.string}\nStart: {tok.start}\nEnd: {tok.end}\nLine: {tok.line.strip()}\n======\n")
            tokenized_code.append(ret)
        return tokenized_code
    except (SyntaxError, tokenize.TokenError, IndentationError, AttributeError):
        return []

In [122]:
subdf = df

In [123]:
subdf['tokenized_source'] = subdf.apply(lambda row: tokenize_code(row['source'], row['cell_type']), axis=1)

In [126]:
subdf.to_json("../data/all-notebooks-tokenized.json", orient='index')